#Charlottesville Fire Department Project: Machine Learning Predictions

Authors: Jackson Barkstrom, Habib Karaky, Josh Schuck, Garrett Vercoe. We joined together the data we used here in the "Cleaning and Merging" code. The data was originally worked on by many, including us, during Civic Innovation Day (special shoutouts to Stephen and Katharine).

Note: We assume a basic understanding of the data we're working with, but we assume only little understanding of machine learning. Our code walks through different machine learning models and their possible utility--the big question, obviously, is which model we should use to predict fires. Obviously, a decision tree is a simple model that we probably don't want to use, but we figured it would be useful for teaching purposes. 

For now, we settled on the Random Forest Regressor because it performed the best out of all of the regression models we somewhat understood and tried--easily beating out models such as bagging and simple decision trees. The random forest is an extremely powerful model, and it produced useful results both as a classifier and as a regressor. We decided we had to use a regression model becuase it allows us to make our own risk categories--it returned decimal risk values between 0 and 1 and we could split these up as we desired. We can easily say "Put anything below .2 in the low risk category, then put everything greater than .6 in the high risk category, then put everything else in the medium risk category. Thus, we' can generate low, medium, and high risk categories. However, we our model really only works for finding the highest-risk homes: the high risk category is the only one that's really significant.

One of the biggest issues we ran into was predicting the fire risk of a building *after it already caught on fire.* Fire risk would go down since the owners of the building would take more safety precautions in the future, no? However, our model didn't look at multiple fires: it only looked at 0 or 1, if the house had a fire at least once since 2003 or if it didn't. We could change this. We recognize that there's some serious danger here of predicting the past (what we're doing) not being the same as predicting the future. 

Despite this shortcoming our models generalize to most houses well. We found pretty clearly that the square footage and the age of buildings are the largest predictors of fire risk, and that makes sense. Really big old houses are way more likely to catch on fire than small new houses. Our models clearly have flaws and cannot predict everything: they could use a lot more data, such as if or if not the homes have smoke alarms installed, and they don't work well for predicting low risk versus medium risk. But they work well for high risk. Our models can predict the homes with the highest risks of fire and the fire department can respond accordingly. Our models aren't the best they could be (we recommend improvement), but they could already be used--right now--to inspect the highest risk homes and decrease fire risk in the Charlottesville community. If our model designates a house as "high risk," the house probably needs attention.

Link to cleaning and merging notebook: https://colab.research.google.com/drive/1EPmKwBAJ560MV5pDJYD1e_iQbE0NezB0#scrollTo=phy9ec8A488x

In [0]:
import pandas as pd
import numpy as np

## Import Data

In [0]:
# Import our joined together data from the "Cleaning and Merging" code

residential = pd.read_csv("https://raw.githubusercontent.com/garrettvercoe/CharlottesvilleFireModel/master/Updated_Residential_Results_cleaned.csv")
commercial = pd.read_csv("https://raw.githubusercontent.com/garrettvercoe/CharlottesvilleFireModel/master/Updated_Commercial_Results_cleaned.csv")

In [3]:
# Examine residential data (feel free not to run this)
residential.head()

,lat,lon,use_type,use_code,style,grade,ext_walls,roof,flooring,bsmt_type,...,full_bathrooms,basement_garage,sq_footage_finished_living,basement,finished_basement,Address,Type,fire_late,fire_early,Fire_final
0,38.064309,-78.488149,C,Duplex,2.0,B,Aluminum,Shingles,Other,Full Basement,...,3.0,0.0,1388.0,694.0,694.0,243 ZAN RD,Residential,0,0.0,0.0
1,38.027193,-78.503804,R,Single Family,1.0,C,Vinyl,Shingles,Hardwood,Full Basement,...,2.0,0.0,1020.0,1020.0,500.0,117 NORTH BAKER ST,Residential,0,0.0,0.0
2,38.034770,-78.487978,R,Single Family,1.0,C,Vinyl,Shingles,Hardwood,Cellar,...,1.0,0.0,925.0,250.0,0.0,702 WEST ST,Residential,0,0.0,0.0
3,38.039464,-78.500183,R,Single Family,1.5,C,Stucco,Shingles,Hardwood,No Basement,...,1.0,0.0,1491.0,NaN,NaN,418 17TH ST,Residential,0,0.0,0.0
4,-90.000000,-152.644271,R,Single Family-1 Conversion,1.0,C,Wood,Shingles,Hardwood,Full Basement,...,2.0,0.0,878.0,878.0,439.0,402 11TH ST,Residential,0,0.0,0.0


In [4]:
# Examine commercial data (feel free not to run this)
commercial.head()

,lat,lon,use_type,use_code,year_built,gross_area,story_height,number_of_stories,address,Address,Type,fire_late,fire_early,Fire_final
0,38.030637,-78.470637,R,Elementary Sch (Entire),1930.0,815.0,12.0,1.0,"402 11TH ST Charlottesville, VA",402 11TH ST,Commercial,0,0.0,0.0
1,38.063599,-78.488920,C,Commercial,1963.0,1134.0,12.0,1.0,"243 ZAN RD Charlottesville, VA",243 ZAN RD,Commercial,0,0.0,0.0
2,38.027207,-78.503849,R,Mini-warehouse,2000.0,400.0,12.0,1.0,"117 NORTH BAKER ST Charlottesville, VA",117 NORTH BAKER ST,Commercial,0,0.0,0.0
3,38.034784,-78.488041,R,Mini-warehouse,2000.0,21600.0,18.0,2.0,"702 WEST ST Charlottesville, VA",702 WEST ST,Commercial,0,0.0,0.0
4,38.034061,-78.489876,R,Mini-warehouse,2000.0,22376.0,18.0,2.0,"302 8TH ST Charlottesville, VA",302 8TH ST,Commercial,0,0.0,0.0


## Cleaning

In [0]:
# Drop variables we are no longer using for Machine Learning
# Drops latitude, longitude, address, if there was a fire 2003-2016,
# and if there was a fire 2016-. Our 'Fire_final' column still shows
# if there was a fire 2003-2018, and this is what we will train our
# models on. Feel free to modify to your liking.

residential_cleaned = residential.drop(['lat','lon','fire_late','fire_early','Address', 'Type'], 1)
commercial_cleaned = commercial.drop(['lat','lon','fire_late','fire_early','Address', 'address', 'Type'], 1)

In [0]:
# Clean the data to be ready for an algorithm 
# We replace nan NaN with 0 and then made it its own category in case there was a pattern to the NaN values
# We make numerical categories with dataframe["colname"].notnull().astype("category").cat.codes

residential_cleaned = residential_cleaned.replace(np.nan, 0, regex=True)
commercial_cleaned = commercial_cleaned.replace(np.nan, 0, regex=True)


residential_cleaned["use_type"] = residential_cleaned["use_type"].astype("category").cat.codes
residential_cleaned["use_code"] = residential_cleaned["use_code"].astype("category").cat.codes
residential_cleaned["grade"] = residential_cleaned["grade"].astype("category").cat.codes
residential_cleaned["ext_walls"] = residential_cleaned["ext_walls"].astype("category").cat.codes
residential_cleaned["roof"] = residential_cleaned["roof"].astype("category").cat.codes
residential_cleaned["flooring"] = residential_cleaned["flooring"].astype("category").cat.codes
residential_cleaned["bsmt_type"] = residential_cleaned["bsmt_type"].astype("category").cat.codes                                                       
residential_cleaned["heating"] = residential_cleaned["heating"].astype("category").cat.codes
                                                       
commercial_cleaned["use_type"] = commercial_cleaned["use_type"].astype("category").cat.codes                                                       
commercial_cleaned["use_code"] = commercial_cleaned["use_code"].astype("category").cat.codes                                                       



In [28]:
# Examine the data
residential_cleaned.head()

,use_type,use_code,style,grade,ext_walls,roof,flooring,bsmt_type,heating,fireplace,...,no_of_stories,total_rooms,bedrooms,half_bathrooms,full_bathrooms,basement_garage,sq_footage_finished_living,basement,finished_basement,Fire_final
0,1,15,2.0,6,1,9,6,4,6,1.0,...,2.0,9.0,5.0,1.0,3.0,0.0,1388.0,694.0,694.0,0.0
1,3,39,1.0,11,23,9,2,4,3,0.0,...,1.0,5.0,2.0,0.0,2.0,0.0,1020.0,1020.0,500.0,0.0
2,3,39,1.0,11,23,9,2,3,3,0.0,...,1.0,5.0,3.0,0.0,1.0,0.0,925.0,250.0,0.0,0.0
3,3,39,1.5,11,21,9,2,5,2,0.0,...,1.5,8.0,5.0,0.0,1.0,0.0,1491.0,0.0,0.0,0.0
4,3,41,1.0,11,24,9,2,4,3,0.0,...,1.0,7.0,3.0,0.0,2.0,0.0,878.0,878.0,439.0,0.0


In [8]:
# Examine the data
commercial_cleaned.head()

,use_type,use_code,year_built,gross_area,story_height,number_of_stories,Fire_final
0,3,21,1930.0,815.0,12.0,1.0,0.0
1,1,12,1963.0,1134.0,12.0,1.0,0.0
2,3,44,2000.0,400.0,12.0,1.0,0.0
3,3,44,2000.0,21600.0,18.0,2.0,0.0
4,3,44,2000.0,22376.0,18.0,2.0,0.0


## Train Test Split

Note: running the split in conjunction with the first cell is does not get rid of variables, which we used because it produces the most accurate model. However, there is a chance that this model will be overfitted. We made the second cell give the model only four variables, just to show how important those 4 variables are in predicting fires, and to show that our model can produce powerful insights on only four variables.

We used 50% of the residential data and 50% of the commercial data to train, but this can easily be changed by editing test_size. That way, if our model tests well among the other 50% of the data, it has high validity.

This first one uses all of the variables. We used it in our final model. Although overfitting is dangerous, there are few variables relative to the size of the dataset. 

In [0]:
# train test split (so that we can validate our model)
from sklearn.model_selection import train_test_split

residential_cleaned_split = residential_cleaned
commercial_cleaned_split = commercial_cleaned

This second one gets rid of variables. Don't run it if you want to look at all the variables.

For our simplest models we used only four variables and found good results in finding the highest risk homes. For residential data we used 1) square footage, 2) year built, 3) does it have a basement, and 4) how many total rooms does it have. For commercial data we used 1) square footage, 2) year built, 3) use code, and 4) number of stories. We found these were the most significant. The final regression model will definitely need more than just these four variables.

In [0]:
# get rid of all but the most useful variables
residential_cleaned_split = residential_cleaned[["sq_footage_finished_living", "year_built", "basement", "total_rooms", "Fire_final"]]
commercial_cleaned_split = commercial_cleaned[["gross_area", "year_built", "use_code", "number_of_stories", "Fire_final"]]

Basic train test split with 50% train 50% split, using data from one of the above three cells

In [0]:
# train test split
from sklearn.model_selection import train_test_split

# train/test split the shortened residential data set
residential_train, residential_test = train_test_split(residential_cleaned_split, test_size = 0.5)

# split into x and y
residential_test_x = residential_test.drop('Fire_final', 1)
residential_test_y = residential_test['Fire_final']

residential_train_x = residential_train.drop('Fire_final', 1)
residential_train_y = residential_train["Fire_final"]

# train/test split the shortened commercial data set
commercial_train, commercial_test = train_test_split(commercial_cleaned_split, test_size = 0.5)

# split into x and y
commercial_test_x = commercial_test.drop('Fire_final', 1)
commercial_test_y = commercial_test['Fire_final']

commercial_train_x = commercial_train.drop('Fire_final', 1)
commercial_train_y = commercial_train["Fire_final"]

In [15]:
# Examine datatypes (just to make check what we're dealing with in our models)
# Everything should say float or int
print(residential_train_x.dtypes)
print(residential_train_y.dtypes)

use_type                         int8
use_code                         int8
style                         float64
grade                            int8
ext_walls                        int8
roof                             int8
flooring                         int8
bsmt_type                        int8
heating                          int8
fireplace                     float64
year_built                    float64
no_of_stories                 float64
total_rooms                   float64
bedrooms                      float64
half_bathrooms                float64
full_bathrooms                float64
basement_garage               float64
sq_footage_finished_living    float64
basement                      float64
finished_basement             float64
dtype: object
float64


In [16]:
# Again, examine datatypes
# Everything should say float or int
print(commercial_train_x.dtypes)
print(commercial_train_y.dtypes)

use_type                int8
use_code                int8
year_built           float64
gross_area           float64
story_height         float64
number_of_stories    float64
dtype: object
float64


## Decision Tree Classifier

First we run our Decision Tree Classifier on the residential data. This is a basic machine learning model that is explained below in the code. It's worth noting that in our case a classifier predicts either 0 or 1, while a regressor will predict a value between 0 and 1. A classifier predicts fires (yes or no, high risk or not), while a regressor predicts more nuanced levels of fire risk (0.2, 0.4, 0.0, 0.9, etc). This decision tree did OK: using only four variables it predicted a little under half of the fires, and ~40% of the time when it said there was a very high risk of fire, there was a fire. Note that 1.0 in the confusion matrix corresponds to fire, and 0 to no fire.

In [17]:
# Our decision tree has "branches" that are based on each variable. For example, our most important variable (with the highest gain)
# is sq_footage_finished_living, so our model might decide to make the left side of the first branch >15k sq feet and the right side 
# <15k sq feet. Essentially, every variable divides the data, with the most important coming first. Google for more explanation. At 
# the smallest divisions in the tree (called "leaf nodes") we will have either a 0 (no probable fire) or 1 (probable fire). 
# Say we're at an arbitrary leaf, and we have 10 addresses from our training data that fall from the top of the tree into this 
# category. If we know 6 of them had fires, every element in this leaf would be predicted as "1" for a fire (60% accuracy on training).

from sklearn.tree import DecisionTreeClassifier

# model
tree = DecisionTreeClassifier(criterion = "entropy")

# train (residential first)
tree.fit(residential_train_x, residential_train_y)

# predict
tree_predictions = tree.predict(residential_test_x)

# This prints the information gain for each feature (very valuable)
print(pd.DataFrame({'Information Gain': tree.feature_importances_}, index = residential_train_x.columns).sort_values('Information Gain', ascending = False))
print("")
confusion = pd.crosstab(residential_test_y, tree_predictions, rownames=['Actual'], colnames = ['Predicted:'], margins = True)

# Note that the success rate isn't a good metric, because obviously our model would be very accurate if it just predicted no fires.
# If we could have 60% accuracy but predict every fire, that's a lot better than 95% accuracy and predicting half of the fires!
print("Total success rate: " + str((confusion.iloc[0,0] + confusion.iloc[1,1]) / confusion.iloc[2,2]))
# This is a confusion matrix, with 1 corresponding ot fire and 0 corresponding to no fire
print(confusion)
print("")
print("When this model said there was high fire risk, there was a fire " + str(confusion.iloc[1,1]/confusion.iloc[2,1]) + " percent of the time.")

                            Information Gain
sq_footage_finished_living          0.227912
year_built                          0.135835
basement                            0.133839
finished_basement                   0.071511
total_rooms                         0.053426
use_type                            0.052415
ext_walls                           0.051993
grade                               0.039973
heating                             0.029106
fireplace                           0.028842
bedrooms                            0.024233
use_code                            0.024175
flooring                            0.022978
full_bathrooms                      0.022418
bsmt_type                           0.019435
roof                                0.018810
style                               0.014786
half_bathrooms                      0.013738
no_of_stories                       0.008229
basement_garage                     0.006345

Total success rate: 0.8958739684921231
Predicted:   0.

In [0]:
# This is a test of cross validation... for our model to be consistent these scores need to be close every time.
# Obviously, this tests on success rate, which is a pretty useless metric for what we're predicting, but it shows
# the presence or absence of consistency.

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(tree, residential_test_x, residential_test_y, cv=10)
print("Scores are: ", scores)

Scores are:  [0.87724551 0.89655172 0.87706147 0.89055472 0.89189189 0.89489489
 0.89339339 0.90840841 0.8963964  0.91591592]


Next, we run our Decision Tree Classifier on the commercial data. This did even better: using only four variables it predicted well over half of the fires, and ~50% of the time when it said there was a very high risk of fire, there was a fire. Note that 1.0 in the confusion matrix corresponds to fire, and 0 to no fire.

In [0]:
# train (commercial second)
tree.fit(commercial_train_x, commercial_train_y)

# predict
tree_predictions = tree.predict(commercial_test_x)

# This prints the information gain for each feature (very valuable)
print(pd.DataFrame({'Information Gain': tree.feature_importances_}, index = commercial_train_x.columns).sort_values('Information Gain', ascending = False))
print("")
confusion = pd.crosstab(commercial_test_y, tree_predictions, rownames=['Actual'], colnames = ['Predicted:'], margins = True)

# Note that the success rate isn't a good metric, because obviously our model would be very accurate if it just predicted no fires.
# If we could have 60% accuracy but predict every fire, that's a lot better than 95% accuracy and predicting half of the fires!
print("Total success rate: " + str((confusion.iloc[0,0] + confusion.iloc[1,1]) / confusion.iloc[2,2]))
# This is a confusion matrix, with 1 corresponding ot fire and 0 corresponding to no fire
print(confusion)
print("")
print("When this model said there was high fire risk, there was a fire " + str(confusion.iloc[1,1]/confusion.iloc[2,1]) + " percent of the time.")

                   Information Gain
gross_area                 0.318979
year_built                 0.312520
use_code                   0.188109
story_height               0.066934
number_of_stories          0.058230
use_type                   0.055227

Total success rate: 0.8825371965544244
Predicted:   0.0  1.0   All
Actual                     
0.0          984  106  1090
1.0           44  143   187
All         1028  249  1277

When this model said there was high fire risk, there was a fire 0.5742971887550201 percent of the time.


In [0]:
# This is a test of cross validation... for our model to be consistent these scores need to be close every time.
# Obviously, this tests on success rate, which is a pretty useless metric for what we're predicting, but it shows
# the presence or absence of consistency.

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(tree, residential_test_x, residential_test_y, cv=10)
print("Scores are: ", scores)

Scores are:  [0.88772455 0.89505247 0.88155922 0.89805097 0.8993994  0.89039039
 0.89189189 0.90690691 0.9024024  0.8963964 ]


## Random Forest Classifier

First we run our Random Forest Classifier on the residential data. A random forest is like a decision tree, but a lot more complex (it's literally just a combination of multiple decision trees) and generally a lot more accurate. It's also explained below in the code. This did very well: using only four variables it predicted about 4/10 of the fires, and *~75-80%* of the time when it said there was a very high risk of fire, there was a fire. Note that 1.0 in the confusion matrix corresponds to fire, and 0 to no fire.

In [18]:
# The Random Forest method introduces more randomness and diversity by applying the bagging method to the feature space. Bagging or Bootstrap Aggregating, 
# consists of randomly sampling subsets of the training data, fitting a model to these smaller data sets, and aggregating the predictions. That is, instead of
# searching greedily for the best predictors to create branches, it randomly samples elements of the predictor space, thus adding more diversity and reducing the 
# variance of the trees at the cost of equal or higher bias.
#
# In plain English, we model decision tree classifiers off of subsets of our data that don't include all the variables. We might have a subset that's just square 
# footage, number of rooms, and number of exterior walls, for example. We have a LOT of different subsets we can take, and each one gets a tree. Then we decide 
# how much weight each tree should get (a tree that uses square footage and year built will be more important than a tree only using roof and the flooring data, 
# since square footage and year built are really important factors). Then, by combining weights of all of these little trees, we get our model (a forest!).
# Again, Google is your friend.

from sklearn.ensemble import RandomForestClassifier

# model
forest =  RandomForestClassifier()

# train (residential first)
forest.fit(residential_train_x, residential_train_y)

# predict
forest_predictions = forest.predict(residential_test_x)

# show feature importances (very valuable, and basically the same meaning to us as information gain in a decision tree)
# shows how important each variable is
print(pd.DataFrame({'Importance': forest.feature_importances_}, index = residential_train_x.columns).sort_values('Importance', ascending = False))
print("")

confusion = pd.crosstab(residential_test_y, forest_predictions, rownames=['Actual'], colnames = ['Predicted:'], margins = True)

# Note that the success rate isn't a good metric, because obviously our model would be very accurate if it just predicted no fires.
# If we could have 60% accuracy but predict every fire, that's a lot better than 95% accuracy and predicting half of the fires!
print("Total success rate: " + str((confusion.iloc[0,0] + confusion.iloc[1,1]) / confusion.iloc[2,2]))
# This is a confusion matrix, with 1 corresponding ot fire and 0 corresponding to no fire
print(confusion)
print("")
print("When this model said there was high fire risk, there was a fire " + str(confusion.iloc[1,1]/confusion.iloc[2,1]) + " percent of the time.")

                            Importance
sq_footage_finished_living    0.173524
year_built                    0.112755
basement                      0.100534
ext_walls                     0.069133
finished_basement             0.065837
total_rooms                   0.064197
grade                         0.042762
bedrooms                      0.039283
heating                       0.036876
full_bathrooms                0.036665
use_type                      0.036511
fireplace                     0.033368
bsmt_type                     0.030573
flooring                      0.026805
use_code                      0.026375
roof                          0.025824
style                         0.024434
no_of_stories                 0.022562
half_bathrooms                0.021161
basement_garage               0.010823

Total success rate: 0.9413353338334584
Predicted:   0.0  1.0   All
Actual                     
0.0         6057   31  6088
1.0          360  217   577
All         6417  248  6665



In [19]:
# This is a test of cross validation... for our model to be consistent these scores need to be close every time.
# Obviously, this tests on success rate, which is a pretty useless metric for what we're predicting, but it shows
# the presence or absence of consistency.

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(forest, residential_test_x, residential_test_y, cv=10)
print("Scores are: ", scores)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Scores are:  [0.94152924 0.93853073 0.94752624 0.93553223 0.94152924 0.93553223
 0.94302849 0.94294294 0.93383459 0.94135338]


Next, we run our Random Forest Classifier on the commercial data. This also did very well: using only four variables it predicted a well over half of the fires (better than residential), and ~60% (worse than residential) of the time when it said there was a very high risk of fire, there was a fire. Note that 1.0 in the confusion matrix corresponds to fire, and 0 to no fire. 

In [24]:
# train (commercial second)
forest.fit(commercial_train_x, commercial_train_y)

# predict
forest_predictions = forest.predict(commercial_test_x)

# show feature importances (very valuable, and basically the same meaning to us as information gain in a decision tree)
# shows how important each variable is
print(pd.DataFrame({'Importance': forest.feature_importances_}, index = commercial_train_x.columns).sort_values('Importance', ascending = False))
print("")

confusion = pd.crosstab(commercial_test_y, forest_predictions, rownames=['Actual'], colnames = ['Predicted:'], margins = True)

# Note that the success rate isn't a good metric, because obviously our model would be very accurate if it just predicted no fires.
# If we could have 60% accuracy but predict every fire, that's a lot better than 95% accuracy and predicting half of the fires!
print("Total success rate: " + str((confusion.iloc[0,0] + confusion.iloc[1,1]) / confusion.iloc[2,2]))
# This is a confusion matrix, with 1 corresponding ot fire and 0 corresponding to no fire
print(confusion)
print("")
print("When this model said there was high fire risk, there was a fire " + str(confusion.iloc[1,1]/confusion.iloc[2,1]) + " percent of the time.")

                   Importance
gross_area           0.314905
year_built           0.281418
use_code             0.221812
story_height         0.082402
number_of_stories    0.065105
use_type             0.034358

Total success rate: 217.66666666666666
Predicted:  0.0  0.05  0.06666666666666667  0.075  0.07777777777777778  0.09  \
Actual                                                                         
0.0         653     6                    2      1                    1     1   
1.0          35     0                    1      0                    0     0   
All         688     6                    3      1                    1     1   

Predicted:  0.1  0.15  0.16666666666666666  0.175  ...   0.425  \
Actual                                             ...           
0.0         185     3                    2      1  ...       1   
1.0           6     0                    0      0  ...       0   
All         191     3                    2      1  ...       1   

Predicted:  0.4333

In [21]:
# This is a test of cross validation... for our model to be consistent these scores need to be close every time.
# Obviously, this tests on success rate, which is a pretty useless metric for what we're predicting, but it shows
# the presence or absence of consistency.

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(forest, residential_test_x, residential_test_y, cv=10)
print("Scores are: ", scores)

Scores are:  [0.93253373 0.94002999 0.94302849 0.94152924 0.93703148 0.93853073
 0.94302849 0.94444444 0.93233083 0.93533835]


## Random Forest Regressor

First we run our Random Forest Regressor on the residential data. It's again worth noting that a regressor returns values between 0 and 1 (i.e. 0.226) as opposed to just 0's and 1's. Otherwise, this model is pretty much the same as the Random Forest Classifier. This model worked best for us--better than models such as a decision tree regressor or a bagging regressor. For now this will be our final model (because it is a regressor it allows us to classify houses according to risk, which is much more in line with the task at hand). Using only four variables it was able to do very well at predicting a high risk category (see below), but more variables were useful in distinguishing between medium risk and low risk. See the code and the output for more information. 

Modify the function risk_function to modify how we convert into risk categories. For more explanation of our model see the cell below.

In [22]:
# This works just like the Random Forest Classifier, only it's a Regressor. To explain, I took the description of the Random Forest Classifier and replaced the
# word "Classifier" with "Regressor" (see below). A decision tree regressor (what this is made of) is a decision tree that has decimal numbers between 0 and 1
# at the leaf nodes instead of just 0 or 1. Say we're at an arbitrary leaf, and we have 10 addresses from our training data that fall into this category. 
# If we know 6 of them had fires, every element in this leaf would be probably be predicted as a decently high decimal value (since we have 6/10 fires).
#
# The Random Forest method introduces more randomness and diversity by applying the bagging method to the feature space. Bagging or Bootstrap Aggregating, 
# consists of randomly sampling subsets of the training data, fitting a model to these smaller data sets, and aggregating the predictions. That is, instead of
# searching greedily for the best predictors to create branches, it randomly samples elements of the predictor space, thus adding more diversity and reducing the 
# variance of the trees at the cost of equal or higher bias.
#
# In plain English, we model decision tree regressors off of subsets of our data that don't include all the variables. We might have a subset that's just square 
# footage, number of rooms, and number of exterior walls, for example. We have a LOT of different subsets we can take, and each one gets a tree. Then we decide 
# how much weight each tree should get (a tree that uses square footage and year built will be more important than a tree only using roof and the flooring data, 
# since square footage and year built are really important factors). Then, by combining weights of all of these little trees, we get our model (a forest!).
# Again, Google is your friend.

from sklearn.ensemble import RandomForestRegressor

# model
forest =  RandomForestRegressor()

# train (residential first)
forest.fit(residential_train_x, residential_train_y)

# predict
forest_predictions = forest.predict(residential_test_x)

# This is our risk function, which converts the outputs of this regression into risk categories 
# low = 1, medium = 2, high = 3 (so if the regressor outputted a 0, we get 1, if it outputted 0.1, we get 2, and 0.4 would return 3)
def risk_function(risk):
  if risk < 0.25:
    return 1
  elif risk < 0.6:
    return 2
  else:
    return 3
  
risk_predictions = pd.Series(forest_predictions).apply(risk_function)

# show feature importances 
print(pd.DataFrame({'Importance': forest.feature_importances_}, index = residential_train_x.columns).sort_values('Importance', ascending = False))

# Since we are using a regressor, a confusion matrix is useless. We're going to have to test the model ourselves.
# This test predicts a fire for every single high risk house, then compares it to the actual data with a confusion matrix
def high_risk_test(risk):
  if risk == 3:
    return 2
  else:
    return 0
  
fire_predictions = risk_predictions.apply(high_risk_test)

confusion = pd.crosstab(residential_test_y, np.array(fire_predictions), rownames=['Actual'], colnames = ['Predicted:'], margins = True)
print("")
print("Percentage of homes in the high risk category with fires: " + str(confusion.iloc[1,1]/confusion.iloc[2,1]))

def medium_risk_test(risk):
  if risk == 2:
    return 1
  else:
    return 0

fire_predictions = risk_predictions.apply(medium_risk_test)
confusion = pd.crosstab(residential_test_y, np.array(fire_predictions), rownames=['Actual'], colnames = ['Predicted:'], margins = True)
print("Precentage of homes in the medium risk category with fires: " + str(confusion.iloc[1,1]/confusion.iloc[2,1]))

def low_risk_test(risk):
  if risk == 1:
    return 1
  else:
    return 0

fire_predictions = risk_predictions.apply(low_risk_test)
confusion = pd.crosstab(residential_test_y, np.array(fire_predictions), rownames=['Actual'], colnames = ['Predicted:'], margins = True)
print("Precentage of homes in the low risk category with fires: " + str(confusion.iloc[1,1]/confusion.iloc[2,1]))


                            Importance
sq_footage_finished_living    0.217682
year_built                    0.120668
basement                      0.104528
finished_basement             0.071770
ext_walls                     0.063223
use_type                      0.053938
total_rooms                   0.053242
grade                         0.039636
heating                       0.037002
bedrooms                      0.029496
full_bathrooms                0.029058
fireplace                     0.025246
use_code                      0.024681
roof                          0.023764
half_bathrooms                0.022096
bsmt_type                     0.021700
flooring                      0.019979
style                         0.016305
no_of_stories                 0.014267
basement_garage               0.011720

Percentage of homes in the high risk category with fires: 0.8313253012048193
Precentage of homes in the medium risk category with fires: 0.1281618887015177
Precentage of homes in t

Next, we run the Random Forest Regressor on the commercial data. Even on just four variables our model for the commercial data worked extremely well--as you can see below, there are clear divisions between low medium and high risk. 

In [23]:
# model
forest =  RandomForestRegressor()

# train (residential first)
forest.fit(commercial_train_x, commercial_train_y)

# predict
forest_predictions = forest.predict(commercial_test_x)

# This is our risk function, which converts the outputs of this regression into risk categories 
# low = 1, medium = 2, high = 3 (so if the regressor outputted a 0, we get 1, if it outputted 0.1, we get 2, and 0.4 would return 3)
def risk_function(risk):
  if risk < 0.25:
    return 1
  elif risk < 0.6:
    return 2
  else:
    return 3
  
risk_predictions = pd.Series(forest_predictions).apply(risk_function)

# show feature importances 
print(pd.DataFrame({'Importance': forest.feature_importances_}, index = commercial_train_x.columns).sort_values('Importance', ascending = False))

# Since we are using a regressor, a confusion matrix is useless. We're going to have to test the model ourselves.
# This test predicts a fire for every single high risk house, then compares it to the actual data with a confusion matrix
def high_risk_test(risk):
  if risk == 3:
    return 2
  else:
    return 0
  
fire_predictions = risk_predictions.apply(high_risk_test)

confusion = pd.crosstab(commercial_test_y, np.array(fire_predictions), rownames=['Actual'], colnames = ['Predicted:'], margins = True)
print("")
print("Percentage of homes in the high risk category with fires: " + str(confusion.iloc[1,1]/confusion.iloc[2,1]))

def medium_risk_test(risk):
  if risk == 2:
    return 1
  else:
    return 0

fire_predictions = risk_predictions.apply(medium_risk_test)
confusion = pd.crosstab(commercial_test_y, np.array(fire_predictions), rownames=['Actual'], colnames = ['Predicted:'], margins = True)
print("Precentage of homes in the medium risk category with fires: " + str(confusion.iloc[1,1]/confusion.iloc[2,1]))

def low_risk_test(risk):
  if risk == 1:
    return 1
  else:
    return 0

fire_predictions = risk_predictions.apply(low_risk_test)
confusion = pd.crosstab(commercial_test_y, np.array(fire_predictions), rownames=['Actual'], colnames = ['Predicted:'], margins = True)
print("Precentage of homes in the low risk category with fires: " + str(confusion.iloc[1,1]/confusion.iloc[2,1]))



                   Importance
gross_area           0.301499
year_built           0.274220
use_code             0.217621
story_height         0.082327
number_of_stories    0.079877
use_type             0.044456

Percentage of homes in the high risk category with fires: 0.8098159509202454
Precentage of homes in the medium risk category with fires: 0.058333333333333334
Precentage of homes in the low risk category with fires: 0.052313883299798795


## Outputting Our Results

If you're just trying to look at our process, stop reading now. Hopefully this helped!

We will output our results for residential data using the Random Forest Regressor, but we could use any regression model on either commercial or residential data with modification. Obviously, one would want to modify the train test split, play with cross validation, and optimize the training of a model as much as possible before outputting results.

First, we calculate our risk values (based on our model that we've trained) and append the calculated risk value to our dataframe. Then, we change it to reflect risk categories (1,2, and 3 for low, medium, and high) which can be changed to include more categories if necessary. Then, we output the data for later use. 

In [0]:
residential_cleaned_split.head()

,use_type,use_code,style,grade,ext_walls,roof,flooring,bsmt_type,heating,fireplace,...,no_of_stories,total_rooms,bedrooms,half_bathrooms,full_bathrooms,basement_garage,sq_footage_finished_living,basement,finished_basement,Fire_final
0,1,15,2.0,6,1,9,6,4,6,1.0,...,2.0,9.0,5.0,1.0,3.0,0.0,1388.0,694.0,694.0,0.0
1,3,39,1.0,11,23,9,2,4,3,0.0,...,1.0,5.0,2.0,0.0,2.0,0.0,1020.0,1020.0,500.0,0.0
2,3,39,1.0,11,23,9,2,3,3,0.0,...,1.0,5.0,3.0,0.0,1.0,0.0,925.0,250.0,0.0,0.0
3,3,39,1.5,11,21,9,2,5,2,0.0,...,1.5,8.0,5.0,0.0,1.0,0.0,1491.0,0.0,0.0,0.0
4,3,41,1.0,11,24,9,2,4,3,0.0,...,1.0,7.0,3.0,0.0,2.0,0.0,878.0,878.0,439.0,0.0


In [0]:
# The word "forest" in the first and second lines can be changed to match whatever regression model we are using (in this case it's the random forest regressor), 
# and we used the word forest to designate our model. If we want to output predictions for the commercial data, replace the word "residential" with the word 
# "commercial" in the code below.

forest_predictions = pd.Series(forest.predict(commercial_cleaned_split.drop(["Fire_final"],1)))
commercial["Detailed Risk"] = forest_predictions
risk_predictions = forest_predictions.apply(risk_function)
commercial["Risk Level"] = risk_predictions

predictions = commercial[["lat", "lon", "Address", "Fire_final", "Detailed Risk"]]

In [0]:
# This is how to output/download a csv from collaboratory
# Forces a download when ran

from IPython.display import Javascript
js_download = """
var csv = '%s';

var filename = 'predictions.csv';
var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
if (navigator.msSaveBlob) { // IE 10+
    navigator.msSaveBlob(blob, filename);
} else {
    var link = document.createElement("a");
    if (link.download !== undefined) { // feature detection
        // Browsers that support HTML5 download attribute
        var url = URL.createObjectURL(blob);
        link.setAttribute("href", url);
        link.setAttribute("download", filename);
        link.style.visibility = 'hidden';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
    }
}
""" % predictions.to_csv(index=False).replace('\n','\\n').replace("'","\'")

Javascript(js_download)

<IPython.core.display.Javascript object>